In [2]:
import pandas as pd
import json
import sys
import datetime
import shutil
import os

import datetime


from dotenv import load_dotenv

from google.cloud import storage, bigquery


load_dotenv() # Cargar las variables de entorno desde el archivo .env

True

# Bigquery

In [3]:
REGION = "us-central1"
PROJECT_ID = "constitucion-20210416"
BIGQUERY = "master"
TABLE = "news_eventregistry"

In [4]:
!gcloud config set project $PROJECT_ID
!gcloud auth login # Cloud SDK
!gcloud auth application-default login

Updated property [core/project].
ERROR: (gcloud.auth.login) unrecognized arguments:
  Cloud
  SDK
  To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=qYuPeyASAeTsJP4EZEM2puSMLYKM2E&access_type=offline&code_challenge=2XbkWjrjBSVwvhyUdyau-YDPimLLqi-Jy3_Ua5MUn44&code_challenge_method=S256


Credentials saved to file: [C:\Users\benja\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "constitucion-20210416" was added to 

In [5]:
# Configura la conexión a BigQuery
client = bigquery.Client()

In [8]:
query = """
SELECT 
    title, 
    body, 
    date, 
    url, 
    source.title AS source_title, 
    source.dataType AS source_dataType, 
    source.uri AS source_uri, 
    time, 
    dateTime 
FROM 
    `constitucion-20210416.master.news_eventregistry` 
WHERE 

    EXTRACT(YEAR FROM date) = 2024 """

# Ejecuta la consulta
query_job = client.query(query)

In [9]:
query_job.result()

In [10]:
df = query_job.to_dataframe()

In [11]:
df

,title,body,date,url,source_title,source_dataType,source_uri,time,dateTime
0,La verdadera razón por la que Rai le otorgó la...,Gala Caldirola y Botota Fox se enfrentaron en ...,2024-09-13,https://www.13.cl/programas/ganar-o-servir/mom...,Canal 13,news,13.cl,21:24:54,2024-09-13 21:24:54+00:00
1,Su apoyo incondicional: Conoce a Katherine Hel...,Katherine Helfmann es una empresaria dedicada ...,2024-09-13,https://www.13.cl/programas/ganar-o-servir/not...,Canal 13,news,13.cl,21:24:33,2024-09-13 21:24:33+00:00
2,Papa Francisco dice que Trump y Kamala Harris ...,El Papa Francisco afirmó el viernes que los do...,2024-09-13,https://www.t13.cl/noticia/mundo/papa-francisc...,T13 (teletrece),news,t13.cl,21:25:11,2024-09-13 21:25:11+00:00
3,"5 payas para sacarse el sombrero este 18, segú...",A pocos días para el inicio de Fiestas Patrias...,2024-09-13,https://www.t13.cl/noticia/tendencias/5-payas-...,T13 (teletrece),news,t13.cl,21:24:33,2024-09-13 21:24:33+00:00
4,"""Es muy triste lo que estoy viviendo"": El lame...",La modelo Adriana Barrientos reveló que perdió...,2024-09-13,https://www.t13.cl/noticia/espectaculos/el-lam...,T13 (teletrece),news,t13.cl,21:04:40,2024-09-13 21:04:40+00:00
...,...,...,...,...,...,...,...,...,...
264768,El poder femenino impulsando la revolución Fin...,"Por. María Fernanda Juppet, CEO de CryptoMKT y...",2024-05-06,https://www.theclinic.cl/2024/05/06/el-poder-f...,"The Clinic - Reportajes, noticias, podcast, vi...",news,theclinic.cl,20:43:31,2024-05-06 20:43:31+00:00
264769,"Elvenir, Sentis y Obexol: Sernac tiene en la m...",Más de 20 tipos de publicidades en Instagram y...,2024-05-06,https://www.theclinic.cl/2024/05/06/elvenir-se...,"The Clinic - Reportajes, noticias, podcast, vi...",news,theclinic.cl,14:43:21,2024-05-06 14:43:21+00:00
264770,Comienza el control de identidad en Lo Valledo...,"Desde este lunes, para ingresar al mercado may...",2024-05-06,https://www.theclinic.cl/2024/05/06/comienza-e...,"The Clinic - Reportajes, noticias, podcast, vi...",news,theclinic.cl,14:43:18,2024-05-06 14:43:18+00:00
264771,Encuesta asegura que mayoría de extranjeros re...,"Asimismo, un 72% de los consultados extranjero...",2024-05-06,https://www.theclinic.cl/2024/05/06/extranjero...,"The Clinic - Reportajes, noticias, podcast, vi...",news,theclinic.cl,18:43:20,2024-05-06 18:43:20+00:00


In [13]:
df['title_body'] = "Titulo de la noticia:\n" + df['title'] + "\nCuerpo de la noticia:\n"  + df['body']


In [19]:
import unidecode


In [20]:
import unidecode

df['title_body_normalized'] = df['title_body'].apply(lambda x: unidecode.unidecode(str(x)) if pd.notnull(x) else x)


3873

In [ ]:

df_filtrado = df[df['title_body_normalized'].str.contains('Hermosilla', case=False, regex=False, na=False)]
len(df_filtrado)

In [28]:
df_filtrado = df[df['title_body_normalized'].str.contains('Luis Hermosilla', regex=False, na=False)]
len(df_filtrado)

3174

In [32]:
df_filtrado.drop_duplicates(subset='title', inplace=True)
df_filtrado.dropna(subset='title', inplace=True)
len(df_filtrado)

C:\Users\benja\AppData\Local\Temp\ipykernel_14228\3256084899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop_duplicates(subset='title', inplace=True)
C:\Users\benja\AppData\Local\Temp\ipykernel_14228\3256084899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.dropna(subset='title', inplace=True)


2632

In [33]:
df_filtrado

,title,body,date,url,source_title,source_dataType,source_uri,time,dateTime,title_body,title_body_normalized
266,Página 11 | El Mercurio de Valparaíso,CASO AUDIOS. Al igual que la defensa del aboga...,2024-09-14,https://www.mercuriovalpo.cl/impresa/2024/09/1...,http://www.mercuriovalpo.cl/,news,mercuriovalpo.cl,06:38:38,2024-09-14 06:38:38+00:00,Titulo de la noticia:\nPágina 11 | El Mercurio...,Titulo de la noticia:\nPagina 11 | El Mercurio...
304,Fiscalía incautó nuevos equipos tecnológicos d...,Esta acción se llevó a cabo en el marco de la ...,2024-09-13,https://www.t13.cl/noticia/politica/fiscalia-i...,T13 (teletrece),news,t13.cl,00:25:39,2024-09-13 00:25:39+00:00,Titulo de la noticia:\nFiscalía incautó nuevos...,Titulo de la noticia:\nFiscalia incauto nuevos...
310,Caso Audios: Felipe Ward dejó su cargo como vi...,"Manuel José Errázuriz, Seremi de Vivienda en e...",2024-09-13,https://www.adnradio.cl/2024/09/12/caso-audios...,ADN Radio 91.7 Chile,news,adnradio.cl,00:55:55,2024-09-13 00:55:55+00:00,Titulo de la noticia:\nCaso Audios: Felipe War...,Titulo de la noticia:\nCaso Audios: Felipe War...
315,Nuevas diligencias en investigación a Ángela V...,La operación logró la incautación de una compu...,2024-09-13,https://www.adnradio.cl/2024/09/12/nuevas-dili...,ADN Radio 91.7 Chile,news,adnradio.cl,00:02:18,2024-09-13 00:02:18+00:00,Titulo de la noticia:\nNuevas diligencias en i...,Titulo de la noticia:\nNuevas diligencias en i...
318,Confiscan equipos con los que Ángela Vivanco i...,Funcionarios del OS7 de Carabineros acudieron ...,2024-09-13,https://www.cnnchile.com/pais/confisca-equipos...,CNN,blog,cnnchile.com,00:00:56,2024-09-13 00:00:56+00:00,Titulo de la noticia:\nConfiscan equipos con l...,Titulo de la noticia:\nConfiscan equipos con l...
...,...,...,...,...,...,...,...,...,...,...,...
261710,Caso Audios: Comisión investigadora sostiene q...,"La comisión investigadora de la Cámara, abiert...",2024-05-07,https://www.t13.cl/noticia/nacional/ex-ante/ca...,T13 (teletrece),news,t13.cl,02:43:45,2024-05-07 02:43:45+00:00,Titulo de la noticia:\nCaso Audios: Comisión i...,Titulo de la noticia:\nCaso Audios: Comision i...
262835,Caso Factop: Álvaro y Antonio Jalaff tienen ca...,"Álvaro Jalaff Sanz, empresario. Antonio Jalaff...",2024-05-03,https://www.df.cl/mercados/banca-fintech/caso-...,Diario Financiero,news,df.cl,08:09:21,2024-05-03 08:09:21+00:00,Titulo de la noticia:\nCaso Factop: Álvaro y A...,Titulo de la noticia:\nCaso Factop: Alvaro y A...
263320,"Caso audios: Comisión Investigadora concluye ""...",Desde la instancia indicaron que todos los ant...,2024-05-06,https://www.adnradio.cl/2024/05/06/caso-audios...,ADN Radio 91.7 Chile,news,adnradio.cl,19:18:14,2024-05-06 19:18:14+00:00,Titulo de la noticia:\nCaso audios: Comisión I...,Titulo de la noticia:\nCaso audios: Comision I...
264388,Redes de corrupción y tráfico de influencias: ...,"""Más que un penalista, era un operador del pod...",2024-05-06,https://radio.uchile.cl/2024/05/06/redes-de-co...,"102.5 FM, Radio Universidad de Chile",news,radio.uchile.cl,18:09:36,2024-05-06 18:09:36+00:00,Titulo de la noticia:\nRedes de corrupción y t...,Titulo de la noticia:\nRedes de corrupcion y t...


# Prefilter

In [34]:
print(df_filtrado['title_body'].iloc[0])


Titulo de la noticia:
Página 11 | El Mercurio de Valparaíso
Cuerpo de la noticia:
CASO AUDIOS. Al igual que la defensa del abogado, el organismo que actúa como querellante a nombre del Estado busca conocer todo el contenido y contactos del imputado.

El Consejo de Defensa del Estado (CDE) solicitó oficialmente acceso a los mensajes y chats extraídos del teléfono celular del abogado Luis Hermosilla, principal imputado en el caso Audios, tal como se le permitió conocerlos a la defensa del abogado.

Según informó el organismo, en su calidad de "querellante institucional en representación del interés público" le solicitó a la Fiscalía la "entrega de todos aquellos registros que estén relacionados con eventuales delitos que pudieran dar lugar a su intervención".

"Es decir, todos aquellos cometidos por funcionarios públicos en el ejercicio de sus funciones y/o que puedan provocar perjuicios al patrimonio fiscal, de modo de someterlos a la valoración acabada y rigurosa que requiere el ejerci

In [35]:
df_filtrado.to_csv('data.csv')